# Growing season definition based on Connaughton (1973)

In [1]:
import os
from datetime import datetime, timezone
import pandas as pd
import pooch
import numpy as np

In [2]:
print("Last updated:", datetime.now(tz=timezone.utc))

Last updated: 2022-12-07 14:20:30.340631+00:00


In [3]:
# DATA_DRIVE = "/run/media/nms/Elements"
DATA_DRIVE = "data"

## Valentia Observatory, Co. Kerry

<https://data.gov.ie/dataset/valentia-observatory-daily-data>

In [4]:
URL = "https://cli.fusio.net/cli/climate_data/webdata/dly2275.csv"
SUB_DIR = os.path.join(DATA_DRIVE, "MetEireann")
KNOWN_HASH = "0fc478a2d8e475f965bbf05e26c1efd6fd7f33468e152ce02dc51b042a771e85"
os.makedirs(SUB_DIR, exist_ok=True)
FILE_NAME = "dly2275.csv"

In [5]:
pooch.retrieve(url=URL, known_hash=KNOWN_HASH, fname=FILE_NAME, path=SUB_DIR)

'/run/media/nms/Backup/Documents/Git/ClimAg/ClimAg/data/MetEireann/dly2275.csv'

In [6]:
val = pd.read_csv(
    os.path.join(SUB_DIR, FILE_NAME), skiprows=24, parse_dates=["date"]
)

In [7]:
val.head()

,date,ind,maxtp,ind.1,mintp,igmin,gmin,ind.2,rain,cbl,...,ind.5,hg,sun,glorad,soil,pe,evap,smd_wd,smd_md,smd_pd
0,1942-01-01,0,11.1,0,9.8,0,8.0,0,1.7,1026.2,...,0,32,,,,0.9,1.1,,,
1,1942-01-02,0,11.6,0,10.0,0,8.6,0,4.6,1023.8,...,0,33,,,,0.6,0.8,,,
2,1942-01-03,0,12.1,0,9.8,0,9.4,0,16.7,1014.5,...,0,39,,,,0.3,0.5,,,
3,1942-01-04,0,9.9,0,4.0,0,6.7,0,3.5,1012.0,...,0,37,,,,0.5,0.7,,,
4,1942-01-05,0,4.9,0,1.4,0,0.4,0,0.0,1022.5,...,0,45,,,,0.9,1.1,,,


In [8]:
# keep only min and max air temps
val = val[["date", "maxtp", "mintp"]]

In [9]:
# filter data for 1954 - 1968
val = val[(val["date"].dt.year >= 1954) & (val["date"].dt.year <= 1968)]

In [10]:
val.head()

,date,maxtp,mintp
4383,1954-01-01,5.8,-1.6
4384,1954-01-02,8.8,3.9
4385,1954-01-03,10.6,4.6
4386,1954-01-04,6.6,2.4
4387,1954-01-05,6.5,0.8


In [11]:
# calculate mean air temperature
val["meantp"] = val[["maxtp", "mintp"]].mean(axis=1)

In [12]:
# keep only mean and date columns
val = val[["date", "meantp"]]

In [13]:
# return only mean values above 4, and subtract by 4
val.loc[(val["meantp"] >= 4.0), "tgrowth"] = val["meantp"] - 4.0

In [14]:
val.head()

,date,meantp,tgrowth
4383,1954-01-01,2.10,NaN
4384,1954-01-02,6.35,2.35
4385,1954-01-03,7.60,3.60
4386,1954-01-04,4.50,0.50
4387,1954-01-05,3.65,NaN


In [15]:
# extract data to determine beginning of growing season
st_1 = {}
for y in val["date"].dt.year.unique():
    st_1[y] = val[
        (val["date"] >= f"{y}-01-01") & (val["date"] <= f"{y}-01-28")
    ]

In [16]:
# extract data to determine end of growing season
st_2 = {}
for y in val["date"].dt.year.unique():
    if y < val["date"].dt.year.max():
        st_2[y] = val[
            (val["date"] >= f"{y}-01-01") & (val["date"] <= f"{y + 1}-01-09")
        ]

In [17]:
# sum of temperatures at the beginning
st_1_list = []
for y in st_1.keys():
    st_1_list.append(list(st_1[y]["tgrowth"].cumsum())[-1])
# remove NaN
st_1_list = [n for n in st_1_list if str(n) != "nan"]

In [18]:
# sum of temperatures at the end
st_2_list = []
for y in st_2.keys():
    st_2_list.append(list(st_2[y]["tgrowth"].cumsum())[-1])
# remove NaN
st_2_list = [n for n in st_2_list if str(n) != "nan"]

In [19]:
st_1_list

[95.0,
 58.2,
 76.95,
 93.59999999999998,
 97.45,
 66.25,
 73.19999999999999,
 79.45000000000002,
 91.35000000000001,
 99.99999999999997,
 80.64999999999998,
 77.9,
 112.84999999999998]

In [20]:
st_2_list

[2391.8000000000025,
 2517.350000000001,
 2415.1499999999996,
 2600.9,
 2684.349999999998,
 2415.600000000001,
 2487.649999999999,
 2219.499999999999,
 2510.1000000000004,
 2332.2999999999997,
 2276.65]

In [21]:
# mean sum of temperatures at the beginning of the growing season
np.mean(st_1_list)

84.83461538461538

In [22]:
# mean sum of temperatures at the end of the growing season
np.mean(st_2_list)

2441.0318181818184

## Shannon Airport, Co. Clare

<https://data.gov.ie/dataset/shannon-airport-daily-data>

In [23]:
URL = "https://cli.fusio.net/cli/climate_data/webdata/dly518.csv"
SUB_DIR = os.path.join(DATA_DRIVE, "MetEireann")
KNOWN_HASH = "bf3eaaa3f060f548540cb463c81ef886472eea4d03e011c60f7ac82d82f61bfb"
os.makedirs(SUB_DIR, exist_ok=True)
FILE_NAME = "dly518.csv"

In [24]:
pooch.retrieve(url=URL, known_hash=KNOWN_HASH, fname=FILE_NAME, path=SUB_DIR)

'/run/media/nms/Backup/Documents/Git/ClimAg/ClimAg/data/MetEireann/dly518.csv'

In [25]:
dat = pd.read_csv(
    os.path.join(SUB_DIR, FILE_NAME), skiprows=24, parse_dates=["date"]
)

In [26]:
dat.head()

,date,ind,maxtp,ind.1,mintp,igmin,gmin,ind.2,rain,cbl,...,ind.5,hg,sun,dos,soil,pe,evap,smd_wd,smd_md,smd_pd
0,1945-09-01,0,19.2,0,14.4,0,13.6,0,0.2,1013.8,...,0,27,1.6,0,,1.6,2.3,,,
1,1945-09-02,0,20.7,0,14.8,0,13.4,3,0.0,1018.2,...,0,16,9.3,0,,2.5,3.5,,,
2,1945-09-03,0,23.3,0,11.8,0,9.6,3,0.0,1021.2,...,0,12,11.4,0,,2.9,3.9,,,
3,1945-09-04,0,21.6,0,9.4,0,6.7,3,0.0,1024.2,...,0,18,5.3,0,,2.0,2.7,,,
4,1945-09-05,0,21.3,0,12.2,0,11.3,3,0.0,1024.9,...,0,13,3.6,0,,1.8,2.4,,,


In [27]:
# keep only min and max air temps
dat = dat[["date", "maxtp", "mintp"]]

In [28]:
# filter data for 1954 - 1968
dat = dat[(dat["date"].dt.year >= 1954) & (dat["date"].dt.year <= 1968)]

In [29]:
dat.head()

,date,maxtp,mintp
3044,1954-01-01,4.6,-5.1
3045,1954-01-02,9.9,2.2
3046,1954-01-03,9.3,0.5
3047,1954-01-04,4.4,-3.1
3048,1954-01-05,2.3,-3.2


In [30]:
# calculate mean air temperature
dat["meantp"] = dat[["maxtp", "mintp"]].mean(axis=1)

In [31]:
# keep only mean and date columns
dat = dat[["date", "meantp"]]

In [32]:
# return only mean values above 4, and subtract by 4
dat.loc[(dat["meantp"] >= 4.0), "tgrowth"] = dat["meantp"] - 4.0

In [33]:
dat.head()

,date,meantp,tgrowth
3044,1954-01-01,-0.25,NaN
3045,1954-01-02,6.05,2.05
3046,1954-01-03,4.90,0.90
3047,1954-01-04,0.65,NaN
3048,1954-01-05,-0.45,NaN


In [34]:
# extract data to determine beginning of growing season
st_1 = {}
for y in dat["date"].dt.year.unique():
    st_1[y] = dat[
        (dat["date"] >= f"{y}-01-01") & (dat["date"] <= f"{y}-03-06")
    ]

In [35]:
# extract data to determine end of growing season
st_2 = {}
for y in dat["date"].dt.year.unique():
    st_2[y] = dat[
        (dat["date"] >= f"{y}-01-01") & (dat["date"] <= f"{y}-12-08")
    ]

In [36]:
# sum of temperatures at the beginning
st_1_list = []
for y in st_1.keys():
    st_1_list.append(list(st_1[y]["tgrowth"].cumsum())[-1])
# remove NaN
st_1_list = [n for n in st_1_list if str(n) != "nan"]

In [37]:
# sum of temperatures at the end
st_2_list = []
for y in st_2.keys():
    st_2_list.append(list(st_2[y]["tgrowth"].cumsum())[-1])
# remove NaN
st_2_list = [n for n in st_2_list if str(n) != "nan"]

In [38]:
st_1_list

[126.6,
 102.84999999999998,
 174.09999999999997,
 173.40000000000006,
 150.5,
 119.34999999999998,
 208.50000000000006,
 40.699999999999996,
 207.5,
 162.85,
 111.35000000000001]

In [39]:
st_2_list

[2234.2000000000007,
 2177.049999999999,
 2369.900000000001,
 2259.099999999999,
 2456.3500000000004,
 2241.8500000000013,
 2010.2500000000005,
 2034.9500000000003,
 2293.250000000002,
 2077.7499999999986,
 2203.8500000000017]

In [40]:
np.mean(st_1_list)

143.4272727272727

In [41]:
np.mean(st_2_list)

2214.4090909090914

## Rosslare, Co. Wexford

<https://data.gov.ie/dataset/rosslare-daily-data>

In [42]:
URL = "https://cli.fusio.net/cli/climate_data/webdatac/dly2615.csv"
SUB_DIR = os.path.join(DATA_DRIVE, "MetEireann")
KNOWN_HASH = "e37a2f84e4172329cd178fc7bddb9720ca559c5c0777bf87cd2b5a42af62c226"
os.makedirs(SUB_DIR, exist_ok=True)
FILE_NAME = "dly2615.csv"

In [43]:
pooch.retrieve(url=URL, known_hash=KNOWN_HASH, fname=FILE_NAME, path=SUB_DIR)

'/run/media/nms/Backup/Documents/Git/ClimAg/ClimAg/data/MetEireann/dly2615.csv'

In [44]:
dat = pd.read_csv(
    os.path.join(SUB_DIR, FILE_NAME),
    skiprows=17,
    skipfooter=1,
    parse_dates=["date"],
    engine="python",
)

In [45]:
dat.head()

,date,ind,maxtp,ind.1,mintp,igmin,gmin,ind.2,rain,cbl,...,ddhm,ind.5,hg,soil,pe,evap,smd_wd,smd_md,smd_pd,glorad
0,1956-12-01,0,10.0,0,6.6,0,4.7,3,0.0,1023.6,...,230,0,18,7.800,0.3,0.4,,,,
1,1956-12-02,0,13.4,0,7.9,0,4.6,2,0.0,1027.4,...,250,0,26,8.700,0.5,0.6,,,,
2,1956-12-03,0,12.2,0,8.4,0,7.8,2,0.0,1029.4,...,260,0,20,9.425,0.4,0.5,,,,
3,1956-12-04,0,11.2,0,9.2,0,6.9,0,0.0,1024.7,...,240,0,30,9.400,0.4,0.6,,,,
4,1956-12-05,0,12.8,0,9.5,0,8.0,0,0.3,1017.4,...,230,0,42,9.900,0.7,0.9,,,,


In [46]:
# keep only min and max air temps
dat = dat[["date", "maxtp", "mintp"]]

In [47]:
# filter data for 1954 - 1968
dat = dat[(dat["date"].dt.year >= 1954) & (dat["date"].dt.year <= 1968)]

In [48]:
dat.tail()

,date,maxtp,mintp
4409,1968-12-27,3.8,0.9
4410,1968-12-28,3.0,0.6
4411,1968-12-29,3.6,0.0
4412,1968-12-30,3.9,1.3
4413,1968-12-31,5.9,2.3


In [49]:
# calculate mean air temperature
dat["meantp"] = dat[["maxtp", "mintp"]].mean(axis=1)

In [50]:
# keep only mean and date columns
dat = dat[["date", "meantp"]]

In [51]:
# return only mean values above 4, and subtract by 4
dat.loc[(dat["meantp"] >= 4.0), "tgrowth"] = dat["meantp"] - 4.0

In [52]:
dat.head()

,date,meantp,tgrowth
0,1956-12-01,8.30,4.30
1,1956-12-02,10.65,6.65
2,1956-12-03,10.30,6.30
3,1956-12-04,10.20,6.20
4,1956-12-05,11.15,7.15


In [53]:
# extract data to determine beginning of growing season
st_1 = {}
for y in dat["date"].dt.year.unique():
    if y > 1956:
        st_1[y] = dat[
            (dat["date"] >= f"{y}-01-01") & (dat["date"] <= f"{y}-02-28")
        ]

In [54]:
# extract data to determine end of growing season
st_2 = {}
for y in dat["date"].dt.year.unique():
    if y > 1956:
        st_2[y] = dat[
            (dat["date"] >= f"{y}-01-01") & (dat["date"] <= f"{y}-12-19")
        ]

In [55]:
# sum of temperatures at the beginning
st_1_list = []
for y in st_1.keys():
    st_1_list.append(list(st_1[y]["tgrowth"].cumsum())[-1])
# remove NaN
st_1_list = [n for n in st_1_list if str(n) != "nan"]

In [56]:
# sum of temperatures at the end
st_2_list = []
for y in st_2.keys():
    st_2_list.append(list(st_2[y]["tgrowth"].cumsum())[-1])
# remove NaN
st_2_list = [n for n in st_2_list if str(n) != "nan"]

In [57]:
st_1_list

[171.20000000000002,
 169.80000000000007,
 144.75,
 120.0,
 171.59999999999994,
 8.350000000000001,
 153.79999999999998,
 174.75000000000003,
 151.15,
 101.50000000000001]

In [58]:
st_2_list

[2374.0,
 2250.349999999999,
 2559.1499999999996,
 2373.5499999999984,
 2044.4499999999998,
 2289.1500000000005,
 2059.1499999999983,
 2248.8000000000006,
 2131.950000000001,
 2242.5000000000014]

In [59]:
np.mean(st_1_list)

136.69

In [60]:
np.mean(st_2_list)

2257.3049999999994